# Representación de Texto para Clasificación  
## Bag-of-Words, TF-IDF y Embeddings (con Word2Vec)

Este notebook explica, paso a paso, tres formas clásicas de representar texto para tareas de *machine learning*:

1. **Bag-of-Words (BoW)**  
2. **TF-IDF (Term Frequency – Inverse Document Frequency)**  
3. **Embeddings** (y el rol de **Word2Vec**)

La idea es que el código te sirva para **experimentar**, mientras que el texto en Markdown te guía con la intuición teórica, igual que si fuera una clase teórica + práctica.


## 1. Bag-of-Words (BoW)

### 1.1. ¿Qué es Bag-of-Words?

**Idea central:** representar un documento como un **histograma de palabras**.

Pasos conceptuales:

1. Tomamos un documento de texto (por ejemplo: `"el perro muerde al gato"`).  
2. Lo **tokenizamos** (lo separamos en palabras):  
   `["el", "perro", "muerde", "al", "gato"]`  
3. Construimos un **vocabulario** a partir de todos los documentos del corpus:  
   por ejemplo, con 3 documentos:

   - Doc 1: `"el perro muerde"`  
   - Doc 2: `"el gato duerme"`  
   - Doc 3: `"el perro duerme con el gato"`  

   El vocabulario podría ser:  
   `["el", "perro", "muerde", "gato", "duerme", "con"]`

4. Cada documento se convierte en un **vector de enteros** cuya longitud es el tamaño del vocabulario.  
   - La posición *i* corresponde a la palabra *i* del vocabulario.  
   - El valor es cuántas veces aparece esa palabra en el documento (TF: *term frequency*).

### 1.2. Punto clave

> El tamaño del vector **NO** es el número de palabras del documento,  
> **es el tamaño del vocabulario del corpus**.

Ejemplo de conteos (BoW):

- Doc 1: `"el perro muerde"`  
  - `el`: 1, `perro`: 1, `muerde`: 1, resto: 0  
  - Vector: `[1, 1, 1, 0, 0, 0]`

- Doc 2: `"el gato duerme"`  
  - `el`: 1, `gato`: 1, `duerme`: 1, resto: 0  
  - Vector: `[1, 0, 0, 1, 1, 0]`

- Doc 3: `"el perro duerme con el gato"`  
  - `el`: 2, `perro`: 1, `gato`: 1, `duerme`: 1, `con`: 1  
  - Vector: `[2, 1, 0, 1, 1, 1]`

Estos vectores suelen ser **largos y dispersos** (muchos ceros) cuando el vocabulario es grande.


In [3]:
# 1. Bag-of-Words con scikit-learn

from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

# Corpus de ejemplo (mismo que en la explicación)
docs = [
    "el perro muerde",
    "el gato duerme",
    "el perro duerme con el gato"
]

vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(docs)

# Vocabulario aprendido
vocab = vectorizer.get_feature_names_out()
print("Vocabulario:", vocab)

# Representación BoW como DataFrame para verlo más claro
df_bow = pd.DataFrame(X_bow.toarray(), columns=vocab)
df_bow.index = [f"Doc {i+1}" for i in range(len(docs))]
df_bow

Vocabulario: ['con' 'duerme' 'el' 'gato' 'muerde' 'perro']


,con,duerme,el,gato,muerde,perro
Doc 1,0,0,1,0,1,1
Doc 2,0,1,1,1,0,0
Doc 3,1,1,2,1,0,1


## 2. TF-IDF (Term Frequency – Inverse Document Frequency)

### 2.1. Motivación

Bag-of-Words trata igual todas las palabras. Pero en la práctica:

- Palabras como **"el", "la", "de"** no son informativas (aparecen en casi todos los documentos).  
- Palabras como **"inflación", "muerde", "con"** pueden ser más informativas.

TF-IDF ajusta los conteos para **dar más peso** a las palabras que:

- Son **frecuentes en un documento**, pero  
- **Raras en el corpus completo**.

### 2.2. Definición

Para una palabra \(t\), documento \(d\) y un corpus de \(N\) documentos:

- **TF(t, d)**: frecuencia de la palabra *t* en el documento *d*.  
- **DF(t)**: número de documentos en los que aparece *t*.  
- **IDF(t)** = $ \log \left( \frac{N}{DF(t)} \right) $.  

Luego:

$$ \text{TF-IDF}(t, d) = TF(t, d) \times IDF(t) $$

### 2.3. Ejemplo simplificado con nuestro corpus

Tenemos 3 documentos (N = 3) y el mismo vocabulario:

`["el", "perro", "muerde", "gato", "duerme", "con"]`

Digamos que:

- `el` aparece en los 3 documentos → DF(el) = 3 → IDF(el) = log(3/3) = 0  

- `muerde` aparece solo en 1 documento → DF(muerde) = 1 → IDF(muerde) = log(3/1) ≈ 1.10  

- `con` aparece solo en 1 documento → IDF(con) ≈ 1.10  

- `perro`, `gato`, `duerme` aparecen en 2 documentos → IDF ≈ log(3/2) ≈ 0.40  

Conclusión:

- Palabras muy frecuentes en el corpus (como "el") tienen **IDF bajo** → poca importancia.  
- Palabras raras (como "muerde" o "con") tienen **IDF alto** → más importancia.

El vector TF-IDF tiene la misma forma que BoW, pero en vez de conteos brutos, hay **pesos reales**.


In [4]:
# 2. TF-IDF con scikit-learn

from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

docs = [
    "el perro muerde",
    "el gato duerme",
    "el perro duerme con el gato"
]

tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(docs)

vocab_tfidf = tfidf.get_feature_names_out()
print("Vocabulario:", vocab_tfidf)

df_tfidf = pd.DataFrame(X_tfidf.toarray(), columns=vocab_tfidf)
df_tfidf.index = [f"Doc {i+1}" for i in range(len(docs))]
df_tfidf

Vocabulario: ['con' 'duerme' 'el' 'gato' 'muerde' 'perro']


,con,duerme,el,gato,muerde,perro
Doc 1,0.000000,0.000000,0.425441,0.000000,0.720333,0.547832
Doc 2,0.000000,0.619805,0.481334,0.619805,0.000000,0.000000
Doc 3,0.492038,0.374207,0.581211,0.374207,0.000000,0.374207


## 3. Embeddings y rol de Word2Vec

### 3.1. ¿Qué es un embedding?

En lugar de representar una palabra como:

- Una posición en un vector gigante (BoW), o  
- Un peso en un vector de TF-IDF,

un **embedding** representa cada palabra como un **vector d-dimensional denso**, por ejemplo d = 100 o d = 300.

Ejemplo (puramente ilustrativo, d = 3):

- `perro` → [0.9, 0.1, 0.7]  
- `gato` → [0.92, 0.15, 0.65]  
- `muerde` → [0.4, 0.8, 0.1]  
- `duerme` → [0.35, 0.75, 0.12]  

Puntos clave:

- Palabras **semánticamente similares** tienen vectores parecidos  
  (por ejemplo, `perro` y `gato`).  
- Los vectores son **densos y de baja dimensión** → más compactos que BoW/TF-IDF.

### 3.2. ¿Cómo representar un documento con embeddings?

Un documento es una secuencia de palabras. Si cada palabra tiene su embedding, se puede:

- Hacer el **promedio** de los vectores de sus palabras.  
- Hacer una **suma ponderada**.  
- Usar modelos más complejos (RNN, Transformers) para producir un embedding de todo el documento.

Ejemplo (promedio simple):

Documento: `"el perro muerde"`  
Supongamos embeddings (ficticios) para `el`, `perro`, `muerde`.  
El embedding del documento = promedio de estos tres vectores.

### 3.3. ¿Qué es Word2Vec y qué rol cumple?

**Word2Vec no es el vector, es el modelo que aprende los vectores.**

Word2Vec es un algoritmo de *deep learning* ligero que, dado un gran corpus de texto, aprende embeddings de palabras.

Tiene dos variantes principales:

- **CBOW (Continuous Bag-of-Words)**:  
  Predice una palabra dado su contexto.

- **Skip-gram**:  
  Predice palabras de contexto dado una palabra central.

Durante este proceso de predicción, el modelo ajusta los vectores de forma que:

- Palabras que aparecen en contextos similares terminen con embeddings similares.

### 3.4. Resumen del flujo con embeddings

1. Entrenas (o descargas) un modelo de embeddings (por ejemplo, Word2Vec).  
2. Obtienes un vector d-dimensional para cada palabra del vocabulario.  
3. Representas cada documento combinando los vectores de sus palabras.  
4. Usas esos vectores como entrada a un clasificador (regresión logística, SVM, red neuronal, etc.).


In [5]:
# 3. Ejemplo de código para entrenar Word2Vec con gensim
# Nota: esto es un ejemplo; requiere tener instalado 'gensim'.


from gensim.models import Word2Vec

# Corpus de ejemplo tokenizado (lista de listas de palabras)
sentences = [
    ["el", "perro", "muerde"],
    ["el", "gato", "duerme"],
    ["el", "perro", "duerme", "con", "el", "gato"]
]

# Entrenar un modelo Word2Vec pequeño
model = Word2Vec(
    sentences=sentences,
    vector_size=10,  # dimensión del embedding
    window=2,        # tamaño de la ventana de contexto
    min_count=1,     # incluir todas las palabras (por ser ejemplo pequeño)
    workers=1,
    sg=1             # 1 = Skip-gram, 0 = CBOW
)

# Obtener el embedding de una palabra
print("Embedding de 'perro':")
print(model.wv["perro"])

# Calcular similitud entre palabras
sim_perro_gato = model.wv.similarity("perro", "gato")
print("\nSimilitud entre 'perro' y 'gato':", sim_perro_gato)

# Representar un documento como promedio de embeddings
import numpy as np

def document_embedding(tokens, wv):
    vecs = [wv[w] for w in tokens if w in wv]
    if not vecs:
        return np.zeros(wv.vector_size)
    return np.mean(vecs, axis=0)

doc_tokens = ["el", "perro", "muerde"]
doc_vec = document_embedding(doc_tokens, model.wv)
print("\nEmbedding del documento 'el perro muerde':")
print(doc_vec)

Embedding de 'perro':
[-0.07511582 -0.00930042  0.09538119 -0.07319167 -0.02333769 -0.01937741
  0.08077437 -0.05930896  0.00045162 -0.04753734]

Similitud entre 'perro' y 'gato': -0.10551018

Embedding del documento 'el perro muerde':
[-0.05401909  0.01267396  0.03501464  0.00838214 -0.01046033 -0.04505576
  0.0635127  -0.01248289 -0.02839585  0.00293801]


# =============================================
# 
# Cargar 1 PDF + 1 Word → Extraer Texto → TF-IDF
# =============================================

In [6]:
import pdfplumber
from docx import Document
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

print("Librerías cargadas correctamente.")

Librerías cargadas correctamente.


In [7]:
# -------------------------------------------------
# RUTAS DE PRUEBA
# -------------------------------------------------

PDF_PATH = "ejemplo.pdf"      #
WORD_PATH = "ejemplo.docx"    # 


In [8]:
# -------------------------------------------------
# Leer PDF (solo PDFs con texto digital)
# -------------------------------------------------
def leer_pdf_texto(path_pdf):
    texto = ""
    with pdfplumber.open(path_pdf) as pdf:
        for page in pdf.pages:
            contenido = page.extract_text()
            if contenido:
                texto += contenido + "\n"
    return texto.strip()


# -------------------------------------------------
# Leer Word (.docx)
# -------------------------------------------------
def leer_word(path_docx):
    doc = Document(path_docx)
    texto = "\n".join([p.text for p in doc.paragraphs])
    return texto.strip()


In [9]:
# -------------------------------------------------
# EXTRAER TEXTO DE LOS DOS DOCUMENTOS
# -------------------------------------------------

texto_pdf = leer_pdf_texto(PDF_PATH)
texto_word = leer_word(WORD_PATH)

print("PDF extraído:")
print(texto_pdf[:1500], "...\n")

print("Word extraído:")
print(texto_word[:1500], "...\n")

PDF extraído:
Documento de Ejemplo Extenso - PDF
Este es un documento PDF más largo, creado para demostrar la extracción de texto desde
archivos PDF que contienen texto nativo. Su objetivo es permitir que los estudiantes del curso
verifiquen cómo los modelos de Machine Learning procesan documentos reales o simulados antes
de convertirlos en representaciones numéricas como TF–IDF.
El contenido incluye un análisis ficticio del comportamiento legislativo en una nación imaginaria,
considerando estadísticas de proyectos de ley, indicadores de eficiencia parlamentaria y métricas
de calidad regulatoria. Asimismo, se mencionan las dificultades comunes en la digitalización de
documentos, la necesidad de sistemas OCR robustos y los desafíos técnicos asociados al
procesamiento de archivos provenientes de múltiples formatos como CSV, PDF, Word y bases de
datos externas.
Se discuten también temas relevantes como el uso de modelos de clasificación para categorizar
artículos de prensa, la construcció

In [16]:
# -------------------------------------------------
# ARMAR DATASET
# -------------------------------------------------

df = pd.DataFrame({
    "texto": [texto_pdf, texto_word],
    "etiqueta": ["pdf_demo", "word_demo"]   # etiquetas ejemplo
})

df


,texto,etiqueta
0,Documento de Ejemplo Extenso - PDF\nEste es un...,pdf_demo
1,Documento de Ejemplo Extenso - Word\nEste es u...,word_demo


In [17]:
# -------------------------------------------------
# LIMPIEZA SUAVE
# -------------------------------------------------

def limpiar(texto):
    texto = texto.lower().replace("\n", " ")
    return texto

df["texto_limpio"] = df["texto"].apply(limpiar)

df

,texto,etiqueta,texto_limpio
0,Documento de Ejemplo Extenso - PDF\nEste es un...,pdf_demo,documento de ejemplo extenso - pdf este es un ...
1,Documento de Ejemplo Extenso - Word\nEste es u...,word_demo,documento de ejemplo extenso - word este es un...


In [12]:
# Lista personalizada de stopwords en español
spanish_stopwords = [
    "de", "la", "que", "el", "en", "y", "a", "los", "del", "se", "las", "por", "un",
    "para", "con", "no", "una", "su", "al", "lo", "como", "más", "pero", "sus", "le",
    "ya", "o", "este", "sí", "porque", "esta", "entre", "cuando", "muy", "sin", "sobre",
    "también", "me", "hasta", "hay", "donde", "quien", "desde", "todo", "nos", "durante",
    "todos", "uno", "les", "ni", "contra", "otros", "ese", "eso", "ante", "ellos", "e", 
    "esto", "mí", "antes", "algunos", "qué", "unos", "yo", "otro", "otras", "otra",
    "él", "tanto", "esa", "estos", "mucho", "quienes", "nada", "muchos", "cual", "poco",
    "ella", "estaré"
]


vectorizer = TfidfVectorizer(
    stop_words=spanish_stopwords,
    ngram_range=(1, 2),     # unigrams + bigrams (muy útil en documentos legales)
    max_features=3000       # tamaño razonable para demos
)

X = vectorizer.fit_transform(df["texto_limpio"])
print("Shape matriz TF-IDF:", X.shape)


Shape matriz TF-IDF: (2, 430)


In [13]:
feature_names = vectorizer.get_feature_names_out()
print(feature_names)


['analiza' 'analiza comportamiento' 'análisis' 'análisis automatizado'
 'análisis ficticio' 'aplicado' 'aplicado procesamiento' 'apunta'
 'apunta ningún' 'archivos' 'archivos docx' 'archivos pdf'
 'archivos provenientes' 'artificial' 'artificial aplicado' 'artículos'
 'artículos prensa' 'asegurar' 'asegurar experimentos' 'asesoría'
 'asesoría parlamentaria' 'asimismo' 'asimismo mencionan'
 'asimismo revisan' 'asociados' 'asociados procesamiento' 'automatizado'
 'automatizado dentro' 'automática' 'automática mediante' 'años'
 'años considerando' 'bases' 'bases datos' 'biblioteca'
 'biblioteca congreso' 'calidad' 'calidad regulatoria' 'capturar'
 'capturar patrones' 'categorizar' 'categorizar artículos' 'cinco'
 'cinco años' 'clasificación' 'clasificación automática'
 'clasificación categorizar' 'complejas' 'complejas lenguaje'
 'comportamiento' 'comportamiento económico' 'comportamiento legislativo'
 'comunes' 'comunes digitalización' 'congreso' 'congreso documento'
 'congreso nacional'

In [14]:
import pandas as pd

df_tfidf = pd.DataFrame(X.toarray(), columns=feature_names)
df_tfidf.head()   # primeras 5 filas


,analiza,analiza comportamiento,análisis,análisis automatizado,análisis ficticio,aplicado,aplicado procesamiento,apunta,apunta ningún,archivos,...,vectores suficientemente,verifiquen,verifiquen cómo,word,word bases,word es,áreas,áreas salud,últimos,últimos cinco
0,0.000000,0.000000,0.045694,0.000000,0.064221,0.000000,0.000000,0.000000,0.000000,0.091388,...,0.064221,0.064221,0.064221,0.045694,0.064221,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.061839,0.061839,0.043999,0.061839,0.000000,0.061839,0.061839,0.061839,0.061839,0.043999,...,0.000000,0.000000,0.000000,0.043999,0.000000,0.061839,0.061839,0.061839,0.061839,0.061839


In [18]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X, df["etiqueta"])

print("Predicción del PDF:", clf.predict(X)[0])
print("Predicción del Word:", clf.predict(X)[1])


Predicción del PDF: pdf_demo
Predicción del Word: word_demo
